In [1]:
import random
import itertools

import matplotlib
import numpy as np

%matplotlib inline

from matrix import Matrix, print_table

In [2]:
A = np.array([[1,2,3],[3,7,7],[0,5,1]], dtype=np.uint8)
B = np.array([[1,2,5],[3,12,4],[0,5,0]], dtype=np.uint8)

In [3]:
A @ B

array([[  7,  41,  13],
       [ 24, 125,  43],
       [ 15,  65,  20]], dtype=uint8)

In [4]:
(A == B).all()

False

In [5]:
np.random.randint(3, size=(3, 3), dtype=np.uint8)

array([[2, 1, 2],
       [2, 2, 1],
       [1, 2, 1]], dtype=uint8)

In [6]:
np.array([[1,2,3],[3,7,7],[0,5,1]], dtype=np.uint8).reshape(-1)

array([1, 2, 3, 3, 7, 7, 0, 5, 1], dtype=uint8)

In [7]:
hash(tuple(np.array([[1,2,3],[3,7,7],[0,5,1]], dtype=np.uint8).reshape(-1)))

6413635302638457342

In [8]:
pow(3, 9)

19683

## The reference (slow) version

In [9]:
def one_step(s):

    def mat_mul(i):
        return i[0] * i[1]

    products = itertools.product(s, s)
    new_s = map(mat_mul, products)
    return s.union(new_s)

A = Matrix(idx = 2)
B = Matrix(idx = 5)
C = Matrix(idx = 100)
result = one_step({A, B, C})

sorted(list(map(lambda x:x.idx(), result)))

[1, 2, 5, 7, 11, 14, 100]

In [10]:
def floodfill(s):
    last = s
    for i in range(100000):
        print(f'step #{i}, {len(last)}')
        new = one_step(last)
        if new == last or len(new) == pow(3, 9):
            last = new
            break
        last = new
    print(f'done, {len(last)}')
    return set(map(lambda x:x.idx(), last))

In [11]:
A = Matrix(idx = 2)
print(A)
B = Matrix(idx = 100)
print(B)
C = Matrix(idx = 1001)
print(C)
s = {A, B, C}

result = floodfill(s)
print(result)
print()
print_table(result)

[[2 0 0]
 [0 0 0]
 [0 0 0]]
[[1 0 2]
 [0 1 0]
 [0 0 0]]
[[2 0 0]
 [1 0 1]
 [1 0 0]]
step #0, 3
step #1, 9
step #2, 19
done, 19
{1, 2, 100, 8038, 14375, 13862, 1001, 1513, 11, 271, 8335, 19, 308, 1459, 758, 532, 56, 731, 28}

011000000001000000010000000010000000000000000000000000001000000000000000000000000
000000000000000000010000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000010000000000000000000000000000000000001000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000010000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000
0010000000000000000000000000010000000

## Multi-processing acceleratesfor 64-core CPUs

In [12]:
from multiprocess import one_step_by_idx

def floodfill_by_idx(s):
    last = s
    for i in range(100000):
        print(f'step #{i}, {len(last)}')
        new = one_step_by_idx(last)
        if new == last or len(new) == pow(3, 9):
            last = new
            break
        last = new
    print(f'done, {len(last)}')
    return last

In [ ]:
s = {1, 5, 10, 21, 50, 100, 600, 1000, 1999, 2001, 4001, 5000, 6000, 7000, 7001, 8000, 9000, 10000, 11000}
result = floodfill_by_idx(s)
# print(result)
print()
print_table(result)

step #0, 19
step #1, 292
step #2, 7967


## What is the index of Identity matrix?

In [ ]:
I = Matrix(idx = 6643)
print(I)